### Resolvendo um problema de negócio


Embora tenha-se falado um pouco sobre validação na etapa anterior, até agora falamos apenas do comportamento de modelos isolados e não de decisões.

Um negócio é feito de decisões e é preciso definir como os dados serão usados para ajudar na tomada de decisão. Apesar de termos contruído três modelos até aqui, o intuito é apenas um: como decidir quais novos clientes aceitar? Para tal são tomadas duas decisões, na verdade: a aprovação do crédito e o limite inicial que será dado.

Você pode encontrar mais sobre o funcionamento do nubank em [nosso site](https://nubank.com.br/perguntas/) na seção "Sobre o Nubank". É importante salientar que não há uma resposta única e correta para este problema e que ele pode ser abordado de diferentes formas. Sinta-se livre para propor qualquer solução que você acredite ser a melhor.

Sabemos que é um problema complexo e adoraríamos ver abordagens simples, racionais e criativas.

Para a sua análise, considere as seguintes informações (elas não precisam ser obrigatoriametne usadas em sua solução):


|           Métrica                            |Valor |
|:--------------------------------------------:|:----:|
|          Taxa de juros do rotativo           |  17% |
|          Taxa de intercâmbio                 |  5%  |
|            Custo unitário do cartão          |  10  |
|Custo por minuto do serviço de atendimento    |  2.5 |
|           Inflação mensal                    | 0.5% |


A maneira com a qual esses valores relacionam-se com o negócio é a seguinte:

- Taxa de juros do rotativo: os clientes tem a opção de pagar apenas 10% de suas faturas e rolar a dívida para o próximo mês, exemplo: em uma fatura de $1000, paga-se 10\% ($100) e rola $900 ($1000 - $100). A próxima fatura será $900 * (1 + 0.17) = $900 (balanço rolado) + $153 (juros do rotativo).

- Taxa de intercâmbio: é cobrado do vendedor 5% do montante total de uma compra feita por um cliente. Por exemplo, o cliente compra uma TV por $1000 e isso trará uma renda de $50 ao Nubank.

- Custo unitário do cartão: é o valor gasto para manufaturar e enviar o cartão para o cliente.

- Custo por minuto do serviço de atendimento: é o valor médio gasto por minuto que o cliente usa o serviço de atendimento.

- Inflação: é a variação média mensal no preço de uma determinada cesta de bens.


**Considerações**:
- se um cliente incorreu em inadimplência, você pode considerar que a soma do montante da última fatura mais o balanço do rotativo do último mês é a quantidade que nós perderemos com aquele cliente. Exemplo: o $900 + $153 citado acima;
- se um cliente é fraudador, ele dará um prejuízo igual ao limite de crédito concedido.

Há total liberdade para usar como bem entender os modelo que foram feitos na etapa anterior. Para responder essas perguntas deve ser feita uma apresentação na qual é discutida a abordagem, como ela foi validada, as premissas, utilização dos dados históricos e etc. Além disso, deve ser entregue:

- O código utilizado;
- Um csv com a definição de aprovado ou não e com qual linha de crédito inicial para o conjunto de teste;
- Uma apresentação em pdf que descreva a sua abordagem.

Tenha em mente que esta apresentação deve convencer executivos a adotarem a sua abordagem.

## Dados de comportamento


|           Coluna                             |Tipo     | Descrição |
|:--------------------------------------------:|:----:| |
|ids|String|identificador único de um aplicante|
|credit_line|  String | Limite do cartão|
|month|Int| Ordenação dos meses que a pessoa é cliente, sendo 0 o primeiro mês dela como cliente|
|spend|Float|Valor gasto naquele mês|
|revolving_balance|Float|Valor que o cliente não pagou da fatura atual e que irá rolar para a próxima |
|card_request| Int |Se o cliente solicitou uma nova via do cartão (ou a primeira) |
| minutes_cs|Float|Quantidade de minutos utilizados do serviço de atendimento ao consumidor |

## Premissas

Algumas premissas que serão levadas em conta como forma de garantir a fundamentação dos cálculos e modelos.


### Balanço do rotativo
- O pagamento do rotativo será considerado como se tivesse ocorrido 

### Fraude
- Se um cliente é fraudador, ele dará um prejuízo igual ao limite de crédito concedido.
- Tal prejuízo de fraude é contabilizado no seu último mês.
- Os gastos dos meses anteriores serão considerados normalmente.

### Inadimplência
- No caso de inadimplência o prejuízo é a soma do montante da última fatura mais o balanço do rotativo do último mês



In [199]:
import pandas as pd
import numpy as np

In [200]:
data = pd.read_csv('../data/raw/spend_train.csv')
acq_data = pd.read_csv('../data/raw/acquisition_train.csv')

In [201]:
max_month = data.groupby('ids')['month'].transform(max)
data['last_month'] = max_month == data.month
# data['penult_month'] = (data.month - max_month) == -1

In [202]:
fraud_family_ids = acq_data.ids[acq_data.target_fraud == 'fraud_friends_family'].values
fraud_id_ids = acq_data.ids[acq_data.target_fraud == 'fraud_id'].values
fraud_ids = np.concatenate((fraud_family_ids, fraud_others_ids))

In [203]:
data['fraud'] = False
data.loc[(data.ids.isin(fraud_ids)),'fraud'] = True

In [204]:
data['fraud_type'] = pd.np.nan
data.loc[data.ids.isin(fraud_family_ids),'fraud_type'] = 'fraud_friends_family'
data.loc[data.ids.isin(fraud_id_ids),'fraud_type'] = 'fraud_id'

In [205]:
default_ids = acq_data[acq_data.target_default == 1].ids.values

In [206]:
data['default'] = False
data.loc[data.ids.isin(default_ids), 'default'] = True

In [207]:
interest_rate = 0.17
interchange_rate = 0.05
inflation_rate = 0.005
unit_card_cost = 10

In [210]:
# data[(data.fraud_type == 'fraud_id') & (data.last_month)]

In [211]:
# data['fraud_cost'] = data.

In [212]:
data['last_revolving'] = data.groupby('ids')['revolving_balance'].shift(1).fillna(0)

In [213]:
data['last_revolving_interest'] = data.last_revolving * interest_rate

In [214]:
data['bill'] = data.spends + data.last_revolving + data.last_revolving_interest

In [215]:
data['interchange_renenue'] = data.spends * interchange_rate

In [216]:
data['card_cost'] = data.card_request * unit_card_cost

In [217]:
data['paid'] = data.bill - data.revolving_balance

In [218]:
data['cs_cost'] = data.minutes_cs * 2.5

In [235]:
data['total_revenue'] = data.last_revolving_interest + data.interchange_renenue
# remove last month revenue in case of fraud or default
data.loc[(data.default | data.fraud) & data.last_month, 'total_revenue'] = 0

In [233]:
data['fraud_cost'] = data.credit_line * data.fraud * data.last_month
data['default_cost'] = (data.bill + data.revolving_balance) * data.default * data.last_month * ~data.fraud

In [236]:
data['total_cost'] = data.cs_cost + data.card_cost + data.fraud_cost + data.default_cost

In [240]:
data['discount_factor'] = (1 + inflation_rate) ** data.month

In [241]:
data['discounted_total_revenue'] = data.total_revenue / data.discount_factor
data['discounted_total_cost'] = data.total_cost / data.discount_factor

In [246]:
revenue = data.discounted_total_revenue.sum()

In [247]:
cost = data.discounted_total_cost.sum()

In [249]:
profit = revenue - cost

In [250]:
profit / revenue


0.8148138546009099